# Wildfire Prediction

This Jupyter Notebook implements a CNN to predict whether a landscape image represents a wildfire or not. PyTorch will be the main library used for this project. 

Image Specifications:
- Type: `.jpg`
- Dimensions: 250 x 250 pixels
- Horizontal/vertical resolution: 96 dpi
- Bit depth: 24

In [ ]:
# Import libraries

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader